# 第7单元: 通过PyBullet的机器人模拟Advantage Actor Critic(A2C) 🤖️

在这份简短的笔记中, 你将学习如何将A2C和PyBullet结合使用. 并训练智能体行走. 更准确地说是训练蜘蛛(他们说是蚂蚁, 但我认为它是蜘蛛 😆) 🕸️

❓如果你有任何问题, 请在discord的#study-group-unit7频道发帖 👉 https://discord.gg/aYka4Yhff9

🎮 环境:

* AntBulletEnv-v0 🕸️

⬇️ 这是**你将在几分钟内实现的目标**的示例([原始视频下载链接](https://huggingface.co/ThomasSimonini/ppo-SpaceInvadersNoFrameskip-v4/resolve/main/replay.mp4)). ⬇️

In [1]:
%%html
<video autoplay controls><source src='./assets/replay.mp4' type='video/mp4'></video>

💡 我们建议你使用Google Colab, 因为某些环境只适用于Ubuntu. Google Colab的免费版本很适合这个教程. 让我们开始吧! 🚀

## 这份笔记来自深度强化学习课程
![Deep Reinforcement Learning Course.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:

* 📖 研究深度强化学习的**理论和实践.**
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ **在独特的环境中训练智能体.**

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗

在深入研究笔记之前, 你需要:

🔲 📚 [阅读第7单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit7/README.md)

🔲 📚 通过阅读章节**学习Advantage Actor Critic(A2C)** 👉 https://huggingface.co/blog/deep-rl-a2c

### 第0步: 设置GPU 💪

* 为了**更快的训练智能体, 我们将使用GPU,** 选择`修改 > 笔记本设置`
![image.png](./assets/image.png)

* `硬件加速器 > GPU`

![image.png](./assets/image1.png)

### 安装依赖项 🔽
第一步是安装多个依赖项:
* `pybullet`: 包含AntBullet环境 🚶
* `stable-baselines3`: 深度强化学习库.
* `huggingface_sb3`: Stable-baselines3的插件, 用于从Hugging Face Hub 🤗 上下载或者上传模型.
* `huggingface_hub`: 允许任何人使用Hugging Face Hub的仓库.

In [ ]:
!pip install gym==0.24.1
!pip install pybullet
!pip install stable-baselines3 tensorboard
!pip install huggingface_sb3
!pip install huggingface_hub

### 第2步: 导入包 📦

In [ ]:
import gym
import pybullet_envs

from huggingface_sb3 import package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.env_util import make_vec_env

### 第3步: 创建AntBulletEnv-v0环境 🕸️
#### 环境 🎮 
在这个环境中, 智能体需要正确的使用它不同的关节才能正确的行走.

In [ ]:
env_id = 'AntBulletEnv-v0'
# 创建环境.
env = gym.make(env_id)

# 获取状态空间和动作空间的大小.
s_size = env.observation_space.shape[0]
a_size = env.action_space

In [ ]:
print('_' * 5 + '可观察的环境' + '_' * 5, end='\n\n')
print('可观察的环境向量的形状', s_size)
print('随机采样环境', env.observation_space.sample())  # 获得一个随机的可观察环境空间.

In [ ]:
print('_' * 5 + '动作空间' + '_' * 5, end='\n\n')
print('动作空间的形状', a_size)
print('随机动作', env.action_space.sample())  # 获得一个随机的动作.

我们需要[对输入特征进行标准化](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html). 为此, 存在一个修饰器(wrapper), 它将计算输入特征的运行平均值和标准差.

我们还通过添加参数`norm_reward=True`来使用相同的修饰器(wrapper)标准化奖罚.

In [ ]:
env = make_vec_env(env_id, n_envs=4)

# 添加这个装饰器来标准化可观察空间和奖罚.
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.)

### 第4步: 创建A2C模型 🤖️
在这种情况下, 因为我们有一个向量作为输入, 我们将使用一个多层感知机(Multi Layer Perception)作为策略.

为了找到最优参数, 我查看了[Stable-Baselines3团队训练的官方智能体](https://huggingface.co/sb3).

In [ ]:
model = A2C(policy='MlpPolicy',
            env=env,
            learning_rate=0.0096,
            n_steps=8,
            gamma=0.99,
            gae_lambda=0.9,
            ent_coef=0.0,
            vf_coef=0.4,
            max_grad_norm=0.5,
            use_rms_prop=True,
            use_sde=True,
            normalize_advantage=False,
            tensorboard_log='./tensorboard',
            policy_kwargs=dict(log_std_init=2,
                               ortho_init=False),
            verbose=1)

### 第5步: 训练A2C智能体 🏃
* 让我们训练智能体 2,000,000 步, 不要忘记使用Colab的GPU. 这大概需要25~40分钟.

In [ ]:
model.learn(2000000)

In [ ]:
# 当保存智能体时同时保存模型和VecNormalize统计信息.
model.save('a2c-AntBulletEnv-v0')
env.save('vec_normalize.pkl')

### 第6步: 评估智能体 📈
* 现在我们的登月着陆器智能体已经训练好了, 我们需要**检查它的性能**.
* Stable-Baselines3 提供了一个方法`evaluate_policy`来进行评估.
* 现在这种情况下, 我们得到的平均奖励是`2371.90 +/- 16.50`

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# 加载保存的统计信息.
eval_env = DummyVecEnv([lambda: gym.make('AntBulletEnv-v0')])
eval_env = VecNormalize.load('vec_normalize.pkl', eval_env)

# 不要在测试时更新参数.
eval_env.training = False
# 测试时不需要对奖罚进行标准化.
eval_env.norm_reward = False

# 加载智能体.
model = A2C.load('a2c-AntBulletEnv-v0')

mean_reward, std_reward = evaluate_policy(model, env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

### 第7步(不涉及核心内容, 可选): 发布我们训练好的模型到 Hub 上 🔥
现在我们看到经过训练之后得到了很棒的结果, 我们可以通过一行代码发布我们训练的模型到hub🤗上.

这有一个模型卡的例子:

![ModelCard.png](./assets/ModelCard.png)

在底层, Hub使用基于git的仓库(即使你不知道什么是git也不用担心), 这意味着你可以在实验和提高你的智能体以后更新新版本的模型.

通过使用`package_to_hub`, **你可以评估, 记录回放视频, 生成智能体的模型卡并把它发布到hub.**

看这边:

* 你可以**展示我们的作品** 🔥
* 你可以**可视化智能体的活动** 👀
* 你可以**与社区分享其他人也可以使用的智能体** 💾
* 你可以**访问排行榜🏆以查看你的智能体和你同学的智能体相比如何** 👉 https://huggingface.co/spaces/chrisjay/Deep-Reinforcement-Learning-Leaderboard

为了能分享你的模型到社区, 有以下三个步骤需要做:

1⃣️ (如果没有完成)创建一个Hugging Face账户 ➡ https://huggingface.co/join

2⃣️ 登陆账户, 然后你需要保存一个Hugging Face的身份验证令牌(token).

* 创建一个新的具有**写入规则**的令牌(https://huggingface.co/settings/tokens)

![image.png](./assets/image2.png)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

如果你使用IDE, 也可在终端中使用以下命令:

In [ ]:
!huggingface-cli login

3⃣️ 我们现在准备好使用`package_to_hub()`发布我们训练的智能体到 🤗 Hub 🔥.

In [ ]:
package_to_hub(model=model,
               model_name=f'a2c-{env_id}',
               model_architecture='A2C',
               env_id=env_id,
               eval_env=eval_env,
               repo_id=f'ThomasSimonini/a2c-{env_id}',  # 记得修改成你的用户名.
               commit_message='Initial commit')

## 额外的挑战(可选) 🏆
最好的学习方式就是**自己进行尝试**! 为什么不试试`HalfCheetahBulletEnv-v0`? 

在[排行榜](https://huggingface.co/spaces/ThomasSimonini/Lunar-Lander-Leaderboard)中, 你将找到你的智能体的位置. 你想要获得第一吗?

以下是一些实现这个目标的想法:
* 训练更多的时间步
* 尝试不同的超参数. 你可以在 👉 https://huggingface.co/models?other=AntBulletEnv-v0 看到其他同学的超参数
* **发布你训练的新模型**到Hub上 🔥

第8单元见! 🔥
## 不断学习, 不断精彩 🤗 ! 